In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    images, labels = mnist.train.next_batch(256)    
    tf.summary.image("inputs", tf.reshape(images, (256, 28, 28, 1)), max_outputs=6)

with tf.variable_scope("model"):
    weights1 = tf.get_variable("weights1", (784, 100))
    layer1 = tf.nn.relu(images @ weights1)

    weights2 = tf.get_variable("weights2", (100, 100))
    layer2 = tf.nn.relu(layer1 @ weights2)

    weights3 = tf.get_variable("weights3", (100, 10))
    outputs = layer2 @ weights3

with tf.name_scope("kernels"):
    kernels = tf.reshape(weights1, (28, 28, 100))
    kernels = tf.transpose(kernels, perm=[2, 0, 1])
    kernels = tf.expand_dims(kernels, axis=-1)
    tf.summary.image("weights1", kernels, max_outputs=6)

with tf.name_scope("activations"):
    tf.summary.image("activations1", tf.reshape(layer1, (256, 10, 10, 1)), max_outputs=6)
    
with tf.name_scope("accuracy"):
    predictions = tf.argmax(outputs, 1)
    sparse_labels = tf.argmax(labels, 1)
    accuracy = tf.reduce_sum(tf.cast(tf.equal(predictions, sparse_labels), tf.int64)) / 256
    tf.summary.scalar("accuracy", accuracy)

with tf.name_scope("loss"):
    l2_loss = tf.nn.l2_loss(weights1) + 0.1 * tf.nn.l2_loss(weights2) + 0.01 * tf.nn.l2_loss(weights3)
    loss = tf.losses.softmax_cross_entropy(labels, outputs) + l2_loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    train_op = tf.train.AdamOptimizer().minimize(loss)
    
summary_op = tf.summary.merge_all()

writer = tf.summary.FileWriter(logdir="log/")

total_steps = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer.add_graph(sess.graph)
    for step in range(total_steps):
        summary, _ = sess.run([summary_op, train_op])     
        writer.add_summary(summary, step)